In [1]:
import torch
# nn contains all of PyTorch's building blocks for neural networks
from torch import nn

import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
from torchvision import datasets, transforms
from visdom import Visdom
import time


In [2]:
batch_size = 200
learning_rate = 0.01
epochs = 10

In [3]:

train_db = datasets.MNIST('../data', train=True, download=True, 
                   transform=transforms.Compose([
                       transforms.ToTensor(), 
                       transforms.Normalize((0.1307,), (0.3081,))
                       ]))
train_loader = torch.utils.data.DataLoader(train_db,batch_size=batch_size, shuffle=True)
test_db = datasets.MNIST('../data', train=False, download=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(), 
                    #    transforms.Normalize((0.1307,), (0.3081,))
                       ]))
test_loader = torch.utils.data.DataLoader(test_db,batch_size=batch_size, shuffle=True)

train_db, val_db = torch.utils.data.random_split(train_db, [50000, 10000])
train_loader = torch.utils.data.DataLoader(train_db, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_db, batch_size=batch_size, shuffle=True)

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            # torch.nn.Dropout(0.5),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            # torch.nn.Dropout(0.5),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.model(x)

In [5]:
viz = Visdom()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MLP().to(device)


Setting up a new session...


In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
# 监听最小值，当验证集的loss不再下降时，减小学习率，patience=5表示连续5次不下降就减小学习率
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

# 对 epoch 记录，每10个epoch减小学习率，gamma=0.1表示变为0.1倍
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
criteon = nn.CrossEntropyLoss().to(device)

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',
                                                   legend=['loss', 'acc.']))

'test'

In [ ]:
global_step = 0
best_accuracy = 0
best_model_path = 'models/'
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28).to(device)
        data, target = data.to(device), target.to(device)
        logits = model(data)
        loss = criteon(logits, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
    
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in val_loader:
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.to(device)
        logits = model(data)
        test_loss += criteon(logits, target).item()
        # 监听最小值，当验证集的loss不再下降时，减小学习率
        # scheduler.step(test_loss)
        pred = logits.argmax(dim=1)  # get the index of the max log-probability
        correct += pred.eq(target).float().sum().item()

    viz.line([[test_loss, correct / len(val_loader.dataset)]], [global_step], win='val', update='append')
    viz.images(data.view(-1, 1, 28, 28), win='val_x',opts={'title':'val_x'})
    viz.text(str(pred.detach().cpu().numpy()), win='val_pred', opts={'title':'val_pred'})
    
    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    print('\Val set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(val_loader.dataset),
        accuracy))
    
    # if accuracy > best_accuracy:
    #     best_accuracy = accuracy
    #     torch.save(model.state_dict(), best_model_path + f'{epoch}_{accuracy:.4f}_best_model.pth')
    #     print(f"Best model saved with accuracy: {best_accuracy:.4f}")
    

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.299768
Train Epoch: 0 [20000/50000 (40%)]	Loss: 0.431636
Train Epoch: 0 [40000/50000 (80%)]	Loss: 0.213986
\Val set: Average loss: 0.0013, Accuracy: 9220.0/10000 (92.20%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.221547
Train Epoch: 1 [20000/50000 (40%)]	Loss: 0.250048
Train Epoch: 1 [40000/50000 (80%)]	Loss: 0.225614
\Val set: Average loss: 0.0009, Accuracy: 9458.0/10000 (94.58%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.157654
Train Epoch: 2 [20000/50000 (40%)]	Loss: 0.116677
Train Epoch: 2 [40000/50000 (80%)]	Loss: 0.114392
\Val set: Average loss: 0.0008, Accuracy: 9536.0/10000 (95.36%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.108623
Train Epoch: 3 [20000/50000 (40%)]	Loss: 0.140196
Train Epoch: 3 [40000/50000 (80%)]	Loss: 0.111087
\Val set: Average loss: 0.0006, Accuracy: 9627.0/10000 (96.27%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.112550
Train Epoch: 4 [20000/50000 (40%)]	Loss: 0.079489
Train Epoch: 4 [40000/50000 (80%)]	Loss: 0.124655
\Val set: Avera

In [8]:
test_loss = 0
correct = 0
for data, target in test_loader:
    data = data.view(-1, 28*28)
    data, target = data.to(device), target.to(device)
    logits = model(data)
    test_loss += criteon(logits, target).item()
    
    pred = logits.argmax(dim=1)  # get the index of the max log-probability
    correct += pred.eq(target).float().sum().item()

viz.line([[test_loss, correct / len(test_loader.dataset)]], [global_step], win='test', update='append')
viz.images(data.view(-1, 1, 28, 28), win='x')
viz.text(str(pred.detach().cpu().numpy()), win='test_pred', opts={'title':'test_pred'})

test_loss /= len(test_loader.dataset)

print('\Val set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

\Val set: Average loss: 0.0025, Accuracy: 9677.0/10000 (97%)



使用记录

没有加dropout
result:
\Val set: Average loss: 0.0004, Accuracy: 9743.0/10000 (97%)
